In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Display all columns in the console
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df =pd.read_excel("05 CNMG D @ CNMO 100424.xlsx", skiprows=1)
df.head(5)

,QTR,PLAY NUM,g SER,g SER#,g DN,g DIST,g FPOS,g HASH,g R/P,RESULT,g GAIN,OFF PERS,CADENCE,PRE FORM,FORM,H ALGN,MOTION,BACKSET,BACK #,PLAY,O BOX,L / R,FLD / BND,STR,WK,QB,REC,ZONE THROWN,FRONT,O COVERAGE
0,1,1,1,1,1,10,-6,NaN,R,Rush,15,NaN,12,NaN,NaN,ACE,3,5,NaN,NaN,GS WING,ZONE,IZ BRICK,L,BND,BLOCK,BLOCK,12,NaN,PINCH
1,1,2,1,2,1,10,-21,NaN,R,Complete,3,12,NaN,NaN,ACE QUEEN,3,5,6 FLY,NaN,GS SING,PERI R,FLY SWP,R,FLD,BLOCK,BLOCK,12,NaN,NaN,PIRATE
2,1,3,1,3,2,7,-24,NaN,P,Complete,6,10,NaN,DEUCE,ACE,NaN,NaN,GW,NaN,DROPBACK,61,L,FLD,BRAVE,UNION,12,15,MID SHT,NaN,IRONMAN
3,1,4,1,4,3,1,-29,NaN,R,Rush,3,JUMBO,NaN,NaN,THICK,2,3,5,NaN,1 JET,UNDER EMPTY DBL WING SING,SHT YDG,SNEAK,R,FLD,BLOCK,BLOCK,44,NaN,BEAR
4,1,5,1,5,1,10,-33,NaN,R,Rush,-11,12,NaN,NaN,DEUCE,2,4,5 JET,NaN,GW WING SING,TRICK,REVERSE PASS,R,FLD,FLAT,CLIMB,12,NaN,NaN,RAVEN


In [8]:
!pip install dataframe-image

### Report 1

In [92]:
# Step 1: Group by OFF PERS and g R/P. Count the occurrences for each unique combination.
report2 = df.groupby(['OFF PERS', 'g R/P']).size().reset_index(name='count')
group_totals = report2.groupby('OFF PERS')['count'].transform('sum')
# Step 2: Calculate the percentage of each play type (R/P/O/A) within each OFF PERS group.
report2['%'] = round((report2['count'] / group_totals) * 100, 1)

report2['display_value'] = report2.apply(
    lambda row: f"{row['count']} ({row['%']:.1f}%)",
    axis=1
)

# Pivot table
final_report_pivot = pd.pivot_table(
    report2,
    index='g R/P',
    columns='OFF PERS',
    values=['count', '%'],
    fill_value=0
)

# Swapping the top level with the second level of the column index.
final_report_pivot = final_report_pivot.swaplevel(0, 1, axis=1)
# First by OFF PERS (level 0), then by the metric name (level 1).
final_report_pivot.sort_index(axis=1, level=0, inplace=True)

final_report_pivot

OFF PERS    10          11          12           20        JUMBO      
             % count     % count     % count      % count      % count
g R/P                                                                 
O         13.6   3.0  24.0   6.0   0.0   0.0    0.0   0.0    0.0   0.0
P         86.4  19.0  36.0   9.0  20.0   1.0  100.0   1.0    0.0   0.0
R          0.0   0.0  40.0  10.0  80.0   4.0    0.0   0.0  100.0   2.0

In [93]:
def style_zeros(val):
    if isinstance(val, (int, float)) and val == 0:
        return 'color: #777777; font-weight: normal;'
    return 'color: white; font-weight: bold;'

# Final display: styled dark-theme table
styled_report1 = final_report_pivot.style \
    .set_caption("Play Type Tendencies by Offensive Personnel") \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('font-size', '16px'), ('font-weight', 'bold'), ('color', 'white'),
            ('text-align', 'center'), ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'), ('color', 'white'), ('border', '1px solid #555555'),
            ('text-align', 'center'), ('font-weight', 'bold')
        ]},
        {'selector': 'td', 'props': [
            ('background-color', '#222222'), ('border', '1px solid #555555'),
            ('text-align', 'center'), ('color', 'white')
        ]},
        {'selector': 'th.row_heading', 'props': [('font-weight', 'bold')]}
    ]) \
    .format("{:.1f}")

styled_report1

### Report 2

In [87]:
# Step 1: Group by FORM and g R/P. Count the occurrences for each unique combination.
report2 = df.groupby(['FORM', 'g R/P']).size().reset_index(name='count')
group_totals = report2.groupby('FORM')['count'].transform('sum')
# Step 2: Calculate the percentage of each play type (R/P/O/A) within each FORM group.
report2['%'] = round((report2['count'] / group_totals) * 100, 1)

# Pivot table
final_report_pivot = pd.pivot_table(
    report2,
    index='g R/P',
    columns='FORM',
    values=['count', '%'],
    fill_value=0
)

# Swapping the top level with the second level of the column index.
final_report_pivot = final_report_pivot.swaplevel(0, 1, axis=1)
# First by OFF PERS (level 0), then by the metric name (level 1).
final_report_pivot.sort_index(axis=1, level=0, inplace=True)

final_report_pivot

FORM    ACE       ACE BUNCH       ACE KING       ACE QUEEN       DEUCE        \
          % count         % count        % count         % count     % count   
g R/P                                                                          
O      16.0   4.0       0.0   0.0      0.0   0.0       0.0   0.0  16.7   2.0   
P      64.0  16.0       0.0   0.0    100.0   1.0      33.3   1.0  50.0   6.0   
R      20.0   5.0     100.0   1.0      0.0   0.0      66.7   2.0  33.3   4.0   

FORM  DOUBLES       QUADS       QUADS KING        THICK        
            % count     % count          % count      % count  
g R/P                                                          
O         0.0   0.0  33.3   4.0        0.0   0.0    0.0   0.0  
P       100.0   1.0  50.0   6.0      100.0   1.0    0.0   0.0  
R         0.0   0.0  16.7   2.0        0.0   0.0  100.0   2.0

In [88]:
styled_report2 = final_report_pivot.style \
    .set_caption("Play Type Tendencies by Formation") \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('font-size', '16px'), ('font-weight', 'bold'), ('color', 'white'),
            ('text-align', 'center'), ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'), ('color', 'white'), ('border', '1px solid #555555'),
            ('text-align', 'center'), ('font-weight', 'bold')
        ]},
        {'selector': 'td', 'props': [
            ('background-color', '#222222'), ('border', '1px solid #555555'),
            ('text-align', 'center'), ('color', 'white')
        ]},
        {'selector': 'th.row_heading', 'props': [('font-weight', 'bold')]}
    ]) \
    .format("{:.1f}")

styled_report2

### Report 3

In [89]:
# Step 1: Filter for "First Plays of Series"
df_openers = df[df['g SER#'] != 1].copy()

# Step 2: Create FZONES from g FPOS
def get_fzone_from_rules(fpos):
    if 0 <= fpos < 10:
        return 'Goal Line (+10 to 0)'
    if 10 <= fpos < 20:
        return 'Red Zone (+20 to +10)'
    if 20 <= fpos <= 35:
        return 'Fringe Zone (+35 to +20)'
    if -20 <= fpos < 20: # This handles the broader "Minus 20 to Plus 35"
        return 'Open Field (-20 to +35)'
    if -20 <= fpos <= -1:
        return 'Coming Out (-1 to -20)'
    # Catch-all for anything outside the defined ranges
    return 'Other'

df_openers['FZONE'] = df_openers['g FPOS'].apply(get_fzone_from_rules)

# Remove 'Other' category
df_openers = df_openers[df_openers['FZONE'] != 'Other'].copy()

# Step 3: Create D_and_D from g DIST, and calculate the percentages
play_counts = df_openers.groupby(['FZONE', 'g DN', 'g DIST', 'g R/P']).size().reset_index(name='count')
group_totals = play_counts.groupby(['FZONE', 'g DN', 'g DIST'])['count'].transform('sum')
play_counts['%'] = round((play_counts['count'] / group_totals) * 100, 1)

# Pivot table
final_report_pivot = pd.pivot_table(
    play_counts,
    index='g R/P',
    columns=['FZONE', 'g DN', 'g DIST'],
    values=['count', '%'],
    fill_value=0
)

final_report_pivot.columns = final_report_pivot.columns.reorder_levels([1, 2, 3, 0])
final_report_pivot = final_report_pivot.sort_index(axis=1, level=[0, 1, 2])

final_report_pivot

FZONE  Fringe Zone (+35 to +20)                                               \
g DN                          1            2                                   
g DIST                       10           1            5            11         
                              % count      % count      % count      % count   
g R/P                                                                          
O                          20.0   1.0    0.0   0.0    0.0   0.0    0.0   0.0   
P                          60.0   3.0    0.0   0.0  100.0   2.0  100.0   1.0   
R                          20.0   1.0  100.0   1.0    0.0   0.0    0.0   0.0   

FZONE                            Goal Line (+10 to 0)                     \
g DN                     3                          1                      
g DIST     17           1                          1            6          
            % count      % count                    % count      % count   
g R/P                                                                      
O         0.0   0.0    0.0   0.0                  0.0   0.0    0.0   0.0   
P       100.0   1.0    0.0   0.0                  0.0   0.0    0.0   0.0   
R         0.0   0.0  100.0   1.0                100.0   1.0  100.0   1.0   

FZONE                                         Open Field (-20 to +35)        \
g DN                     2                                          1         
g DIST     9            6            9                             10         
            % count      % count      % count                       % count   
g R/P                                                                         
O         0.0   0.0    0.0   0.0    0.0   0.0                   100.0   1.0   
P       100.0   1.0  100.0   1.0  100.0   1.0                     0.0   0.0   
R         0.0   0.0    0.0   0.0    0.0   0.0                     0.0   0.0   

FZONE                                         Red Zone (+20 to +10)        \
g DN        2                                                     1         
g DIST     9            10           12                          10         
            % count      % count      % count                     % count   
g R/P                                                                       
O         0.0   0.0    0.0   0.0    0.0   0.0                   0.0   0.0   
P       100.0   1.0  100.0   1.0  100.0   1.0                 100.0   1.0   
R         0.0   0.0    0.0   0.0    0.0   0.0                   0.0   0.0   

FZONE                
g DN        2        
g DIST     7         
            % count  
g R/P                
O         0.0   0.0  
P       100.0   1.0  
R         0.0   0.0

In [90]:
styled_report3 = final_report_pivot.style \
    .set_caption("Play Type Tendencies by Down & Distance – Non-Openers") \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('font-size', '16px'), ('font-weight', 'bold'), ('color', 'white'),
            ('text-align', 'center'), ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'), ('color', 'white'), ('border', '1px solid #555555'),
            ('text-align', 'center'), ('font-weight', 'bold')
        ]},
        {'selector': 'td', 'props': [
            ('background-color', '#222222'), ('border', '1px solid #555555'),
            ('text-align', 'center'), ('color', 'white')
        ]},
        {'selector': 'th.row_heading', 'props': [('font-weight', 'bold')]}
    ]) \
    .format("{:.1f}")


styled_report3

### Report 4

In [59]:
# Step 1: Filter for "First Plays of Series"
df_openers = df[df['g SER#'] == 1].copy()

# Step 2: Create FZONES from g FPOS
def get_fzone_from_rules(fpos):
    if 0 <= fpos < 10:
        return 'Goal Line (+10 to 0)'
    if 10 <= fpos < 20:
        return 'Red Zone (+20 to +10)'
    if 20 <= fpos <= 35:
        return 'Fringe Zone (+35 to +20)'
    if -20 <= fpos < 20: # This handles the broader "Minus 20 to Plus 35"
        return 'Open Field (-20 to +35)'
    if -20 <= fpos <= -1:
        return 'Coming Out (-1 to -20)'
    # Catch-all for anything outside the defined ranges
    return 'Other'

df_openers['FZONE'] = df_openers['g FPOS'].apply(get_fzone_from_rules)

# Remove 'Other' category
df_openers = df_openers[df_openers['FZONE'] != 'Other'].copy()

# Step 3: Create D_and_D from g DIST, and calculate the percentages
play_counts = df_openers.groupby(['FZONE', 'g DN', 'g DIST', 'g R/P']).size().reset_index(name='count')
group_totals = play_counts.groupby(['FZONE', 'g DN', 'g DIST'])['count'].transform('sum')
play_counts['%'] = round((play_counts['count'] / group_totals) * 100, 1)

# Pivot table
final_report_pivot = pd.pivot_table(
    play_counts,
    index='g R/P',
    columns=['FZONE', 'g DN', 'g DIST'],
    values=['count', '%'],
    fill_value=0
)

# Swapping the top level with the second level of the column index.
final_report_pivot.columns = final_report_pivot.columns.reorder_levels([1, 2, 3, 0])
final_report_pivot = final_report_pivot.sort_index(axis=1, level=[0, 1, 2])

final_report_pivot

FZONE  Fringe Zone (+35 to +20)       Open Field (-20 to +35)      
g DN                          1                             1      
g DIST                       10                            10      
                              % count                       % count
g R/P                                                              
A                           0.0   0.0                    25.0   1.0
O                           0.0   0.0                    25.0   1.0
P                           0.0   0.0                    25.0   1.0
R                         100.0   1.0                    25.0   1.0

In [91]:
styled_report4 = final_report_pivot.style \
    .set_caption("Play Type Tendencies by Down & Distance – Openers") \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('font-size', '16px'), ('font-weight', 'bold'), ('color', 'white'),
            ('text-align', 'center'), ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'), ('color', 'white'), ('border', '1px solid #555555'),
            ('text-align', 'center'), ('font-weight', 'bold')
        ]},
        {'selector': 'td', 'props': [
            ('background-color', '#222222'), ('color', 'white'), ('border', '1px solid #555555'),
            ('text-align', 'center')
        ]},
        {'selector': 'th.row_heading', 'props': [('font-weight', 'bold')]}
    ]) \
    .format("{:.1f}")

styled_report4

### Report 5

In [69]:
# Step 1: Filter to only Run plays
df_run = df[df['g R/P'] == 'R'].copy()

# Step 2: Group by OFF PERS, O BOX, L / R
grouped = df_run.groupby(['OFF PERS', 'O BOX', 'L / R']).size().reset_index(name='count')

# Step 3: Calculate % within each OFF PERS & O BOX
grouped['total'] = grouped.groupby(['OFF PERS', 'O BOX'])['count'].transform('sum')
grouped['%'] = round(grouped['count'] / grouped['total'] * 100, 1)

# Step 4: Pivot with correct column order
pivot = pd.pivot_table(
    grouped,
    index='L / R',
    columns=['OFF PERS', 'O BOX'],
    values=['count', '%'],
    fill_value=0
)

pivot.columns = pivot.columns.reorder_levels([1, 2, 0])
pivot = pivot.sort_index(axis=1, level=[0, 1, 2])

pivot

OFF PERS         11                                                   12  \
O BOX           DUO       GF CT        GT CT       OZ BRICK          GAP   
                  % count     % count      % count        % count      %   
L / R                                                                      
DUO             0.0   0.0   0.0   0.0    0.0   0.0      0.0   0.0  100.0   
FLY SWP         0.0   0.0   0.0   0.0    0.0   0.0      0.0   0.0    0.0   
L             100.0   4.0  50.0   1.0    0.0   0.0     33.3   1.0    0.0   
OZ              0.0   0.0   0.0   0.0    0.0   0.0      0.0   0.0    0.0   
R               0.0   0.0  50.0   1.0  100.0   1.0     66.7   2.0    0.0   
REVERSE PASS    0.0   0.0   0.0   0.0    0.0   0.0      0.0   0.0    0.0   
SHT YDG         0.0   0.0   0.0   0.0    0.0   0.0      0.0   0.0    0.0   

OFF PERS                                                   \
O BOX              PERI R        TRICK         ZONE         
             count      % count      % count      % count   
L / R                                                       
DUO            1.0    0.0   0.0    0.0   0.0    0.0   0.0   
FLY SWP        0.0  100.0   1.0    0.0   0.0    0.0   0.0   
L              0.0    0.0   0.0    0.0   0.0    0.0   0.0   
OZ             0.0    0.0   0.0    0.0   0.0  100.0   1.0   
R              0.0    0.0   0.0    0.0   0.0    0.0   0.0   
REVERSE PASS   0.0    0.0   0.0  100.0   1.0    0.0   0.0   
SHT YDG        0.0    0.0   0.0    0.0   0.0    0.0   0.0   

OFF PERS                         JUMBO        
O BOX        UNDER EMPTY DBL WING SING        
                                     % count  
L / R                                         
DUO                                0.0   0.0  
FLY SWP                            0.0   0.0  
L                                  0.0   0.0  
OZ                                 0.0   0.0  
R                                  0.0   0.0  
REVERSE PASS                       0.0   0.0  
SHT YDG                          100.0   2.0

In [70]:
styled_pivot = pivot.style \
    .set_caption("Run Play Direction by OFF PERS > O BOX") \
    .set_properties(**{
        'background-color': '#222222',
        'color': 'white',
        'border': '1px solid #555',
        'text-align': 'center'
    }) \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('color', 'white'),
            ('font-size', '16px'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'),
            ('color', 'white'),
            ('border', '1px solid #555'),
            ('font-weight', 'bold'),
            ('text-align', 'center')
        ]}
    ]) \
    .format("{:.1f}")

styled_pivot

### Report 6

In [71]:
# Step 1: Filter to only Pun plays
df_pass = df[df['g R/P'] == 'R'].copy()

# Step 2: Group by OFF PERS, O BOX, FLD / BND
# Count the occurrences for each unique combination.
grouped = df_pass.groupby(['OFF PERS', 'O BOX', 'FLD / BND']).size().reset_index(name='count')

# Step 3: Calculate percentage within each OFF PERS group
grouped['total'] = grouped.groupby(['OFF PERS', 'O BOX'])['count'].transform('sum')
grouped['%'] = (grouped['count'] / grouped['total'] * 100).round(2)

# Pivot to match desired format
pivot = grouped.pivot_table(
    index='FLD / BND', 
    columns=['OFF PERS', 'O BOX'], 
    values=['count', '%'],
    fill_value=0
)

pivot.columns = pivot.columns.reorder_levels([1, 2, 0])
pivot = pivot.sort_index(axis=1, level=[0, 1, 2])

pivot

OFF PERS      11                                                    12        \
O BOX        DUO        GF CT        GT CT       OZ BRICK          GAP         
               % count      % count      % count        % count      % count   
FLD / BND                                                                      
BND          0.0   0.0  100.0   2.0    0.0   0.0    100.0   3.0    0.0   0.0   
FLD        100.0   4.0    0.0   0.0  100.0   1.0      0.0   0.0    0.0   0.0   
L            0.0   0.0    0.0   0.0    0.0   0.0      0.0   0.0    0.0   0.0   
R            0.0   0.0    0.0   0.0    0.0   0.0      0.0   0.0  100.0   1.0   
SNEAK        0.0   0.0    0.0   0.0    0.0   0.0      0.0   0.0    0.0   0.0   

OFF PERS                                                             JUMBO  \
O BOX     PERI R        TRICK         ZONE       UNDER EMPTY DBL WING SING   
               % count      % count      % count                         %   
FLD / BND                                                                    
BND          0.0   0.0    0.0   0.0    0.0   0.0                       0.0   
FLD          0.0   0.0    0.0   0.0    0.0   0.0                       0.0   
L            0.0   0.0    0.0   0.0  100.0   1.0                       0.0   
R          100.0   1.0  100.0   1.0    0.0   0.0                       0.0   
SNEAK        0.0   0.0    0.0   0.0    0.0   0.0                     100.0   

OFF PERS         
O BOX            
          count  
FLD / BND        
BND         0.0  
FLD         0.0  
L           0.0  
R           0.0  
SNEAK       2.0

In [72]:
styled_pivot = pivot.style \
    .set_caption("Run Play Direction (Field vs Boundary) by OFF PERS > O BOX") \
    .set_properties(**{
        'background-color': '#222222',
        'color': 'white',
        'border': '1px solid #555',
        'text-align': 'center'
    }) \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('color', 'white'),
            ('font-size', '16px'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'),
            ('color', 'white'),
            ('border', '1px solid #555'),
            ('font-weight', 'bold'),
            ('text-align', 'center')
        ]}
    ]) \
    .format("{:.1f}")

styled_pivot

### Report 7

In [73]:
# Step 1: Filter to only PASS plays
df_pass = df[df['g R/P'] == 'P'].copy()

# Step 2: Group by OFF PERS, O BOX, FLD / BND
# Count the occurrences for each unique combination.
grouped = df_pass.groupby(['OFF PERS', 'O BOX', 'FLD / BND']).size().reset_index(name='count')

# Step 3: Calculate percentage within each OFF PERS group
grouped['total'] = grouped.groupby(['OFF PERS', 'O BOX'])['count'].transform('sum')
grouped['%'] = (grouped['count'] / grouped['total'] * 100).round(2)

# Pivot to match desired format
pivot = grouped.pivot_table(
    index='FLD / BND', 
    columns=['OFF PERS', 'O BOX'], 
    values=['count', '%'],
    fill_value=0
)

pivot.columns = pivot.columns.reorder_levels([1, 2, 0])
pivot = pivot.sort_index(axis=1, level=[0, 1, 2])

pivot

OFF PERS      10                                                            \
O BOX         51           61           62           67           73         
               % count      % count      % count      % count      % count   
FLD / BND                                                                    
BND          0.0   0.0  22.22   2.0    0.0   0.0  100.0   1.0  100.0   1.0   
FLC          0.0   0.0   0.00   0.0  100.0   1.0    0.0   0.0    0.0   0.0   
FLD        100.0   1.0  77.78   7.0    0.0   0.0    0.0   0.0    0.0   0.0   
L            0.0   0.0   0.00   0.0    0.0   0.0    0.0   0.0    0.0   0.0   
R            0.0   0.0   0.00   0.0    0.0   0.0    0.0   0.0    0.0   0.0   

OFF PERS                                              11                    \
O BOX         88       OZ READ        QUICK           61          62         
               % count       % count      % count      % count     % count   
FLD / BND                                                                    
BND        100.0   1.0   100.0   1.0    0.0   0.0  100.0   1.0  50.0   1.0   
FLC          0.0   0.0     0.0   0.0    0.0   0.0    0.0   0.0   0.0   0.0   
FLD          0.0   0.0     0.0   0.0    0.0   0.0    0.0   0.0  50.0   1.0   
L            0.0   0.0     0.0   0.0  100.0   1.0    0.0   0.0   0.0   0.0   
R            0.0   0.0     0.0   0.0    0.0   0.0    0.0   0.0   0.0   0.0   

OFF PERS                                                                       \
O BOX         71           76          77       CROSS 62       CROSS 76         
               % count      % count     % count        % count        % count   
FLD / BND                                                                       
BND          0.0   0.0  100.0   1.0  50.0   1.0    100.0   1.0    100.0   1.0   
FLC          0.0   0.0    0.0   0.0   0.0   0.0      0.0   0.0      0.0   0.0   
FLD        100.0   1.0    0.0   0.0  50.0   1.0      0.0   0.0      0.0   0.0   
L            0.0   0.0    0.0   0.0   0.0   0.0      0.0   0.0      0.0   0.0   
R            0.0   0.0    0.0   0.0   0.0   0.0      0.0   0.0      0.0   0.0   

OFF PERS        12           20        
O BOX     DROPBACK           71        
                 % count      % count  
FLD / BND                              
BND            0.0   0.0    0.0   0.0  
FLC            0.0   0.0    0.0   0.0  
FLD            0.0   0.0  100.0   1.0  
L              0.0   0.0    0.0   0.0  
R            100.0   1.0    0.0   0.0

In [74]:
styled_pivot = pivot.style \
    .set_caption("Pass Play Direction (Field vs Boundary) by OFF PERS > O BOX") \
    .set_properties(**{
        'background-color': '#222222',
        'color': 'white',
        'border': '1px solid #555',
        'text-align': 'center'
    }) \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('color', 'white'),
            ('font-size', '16px'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'),
            ('color', 'white'),
            ('border', '1px solid #555'),
            ('font-weight', 'bold'),
            ('text-align', 'center')
        ]}
    ]) \
    .format("{:.1f}")

styled_pivot

### Report 8

In [98]:
report8 = df[df['g R/P'].isin(['P', 'O', 'A'])].copy()
report8['FORM'] = report8['FORM'].fillna('(blank)')

# Step 1: Unpivot STR/WK into 'side' + 'concept'
df_long = pd.melt(
    report8,
    id_vars=['FORM', 'g R/P'],
    value_vars=['STR', 'WK'],
    var_name='side',        # 'STR' or 'WK'
    value_name='concept'    # actual concept name
)

# Step 2: Drop rows with no concept
df_long = df_long[df_long['concept'].notna() & (df_long['concept'] != '')]

# Step 3: Count and % per (FORM, g R/P, side)
concept_counts = df_long.groupby(['concept', 'side', 'FORM', 'g R/P']).size().reset_index(name='count')
concept_counts['total'] = concept_counts.groupby(['FORM', 'g R/P', 'side'])['count'].transform('sum')
concept_counts['%'] = round(concept_counts['count'] / concept_counts['total'] * 100, 1)

# Step 4: Pivot with 'concept' and 'side' as index
pivot = pd.pivot_table(
    concept_counts,
    index=['side', 'concept'],
    columns=['FORM', 'g R/P'],
    values=['count', '%'],
    fill_value=0
)

# Step 5: Reorder column levels: FORM > g R/P > metric
pivot.columns = pivot.columns.reorder_levels([1, 2, 0])
pivot = pivot.sort_index(axis=1, level=[0, 1, 2])

pivot

FORM               (blank)                                    ACE              \
g R/P                    A            O            P            O           P   
                         % count      % count      % count      % count     %   
side concept                                                                    
STR  A SCREEN          0.0   0.0    0.0   0.0    0.0   0.0   50.0   2.0   0.0   
     BLOCK             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     BND             100.0   1.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     BRAVE             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0  26.7   
     BUBBLE            0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     DIVIDE            0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.7   
     DRIVE             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     DUCK              0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.7   
     FISH              0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.7   
     FLAT              0.0   0.0    0.0   0.0    0.0   0.0   25.0   1.0   0.0   
     FLD               0.0   0.0    0.0   0.0  100.0   1.0    0.0   0.0   6.7   
     FLOOD             0.0   0.0    0.0   0.0    0.0   0.0   25.0   1.0   6.7   
     HOTEL             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.7   
     OHIO              0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     OZ READ           0.0   0.0  100.0   1.0    0.0   0.0    0.0   0.0   0.0   
     R                 0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     RACE              0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.7   
     RB BUBBLE         0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     SMASH             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0  13.3   
     STICK             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0  13.3   
     UNION             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     WHEEL             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
WK   A SCREEN N GO     0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     B SCREEN n GO   100.0   1.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     BLOCK             0.0   0.0    0.0   0.0    0.0   0.0  100.0   4.0   0.0   
     BRAVE             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.2   
     DAGGER            0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.2   
     DICE              0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.2   
     FISH              0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     FLD               0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     FLOOD             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     HOTEL             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.2   
     JAILBREAK         0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     L                 0.0   0.0  100.0   1.0    0.0   0.0    0.0   0.0   0.0   
     LEVELS            0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.2   
     OHIO              0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     PUSH              0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.2   
     RACE              0.0   0.0    0.0   0.0   50.0   1.0    0.0   0.0   6.2   
     SEATTLE           0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     SLANT             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   0.0   
     SMASH             0.0   0.0    0.0   0.0   50.0   1.0    0.0   0.0  12.5   
     STICK             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.2   
     UNION             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0  31.2   
     WHEEL             0.0   0.0    0.0   0.0    0.0   0.0    0.0   0.0   6.2   

FORM                     ACE KING       ACE QUEEN       DEUCE              \
g R/P                           P 

In [99]:
styled_pivot = pivot.style \
    .set_caption("STR/WK Concept Distribution by FORM > Play Type") \
    .set_properties(**{
        'background-color': '#222222',
        'color': 'white',
        'border': '1px solid #555',
        'text-align': 'center'
    }) \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('color', 'white'),
            ('font-size', '16px'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'),
            ('color', 'white'),
            ('border', '1px solid #555'),
            ('font-weight', 'bold'),
            ('text-align', 'center')
        ]}
    ]) \
    .format("{:.1f}")

styled_pivot

### Report 9

In [79]:
# Step 1: Filter passing plays
# Assuming 'df' is your full play-level DataFrame
pass_results = ['Complete', 'Incomplete', 'Interception']
passing_df = df[df['RESULT'].isin(pass_results)].copy()

# Step 2: Group by QB, Receiver, Zone, and Coverage
grouped = passing_df.groupby(['QB', 'REC', 'O COVERAGE', 'ZONE THROWN'])

# Step 3: Count attempts and completions
summary_df = grouped.agg(
    attempts=('RESULT', 'count'),
    completions=('RESULT', lambda x: (x == 'Complete').sum())
).reset_index()

# Step 4: Calculate completion rate
summary_df['%'] = summary_df['completions'] / summary_df['attempts']

# rename
summary_df.rename(columns={'attempts': 'count', 'completion_pct': '%'}, inplace=True)

# Pivot table
pivot = pd.pivot_table(
    summary_df,
    index=['QB', 'REC'],
    columns=['O COVERAGE', 'ZONE THROWN'],
    values=['count', '%'], 
    fill_value=0
)

# Reorder the Column Levels for Readability ---
# Original order: (0:metric, 1:O COVERAGE, 2:ZONE THROWN)
# New order:      (1:O COVERAGE, 2:ZONE THROWN, 0:metric)
pivot = pivot.reorder_levels([1, 2, 0], axis=1)

# Sort the columns to ensure a logical order (alphabetical/numerical)
pivot.sort_index(axis=1, inplace=True)
pivot

O COVERAGE    DODGE                                      FORD                \
ZONE THROWN BND SHT       FLD SHT       INT SHT       BND SHT       FLD INT   
                  % count       % count       % count       % count       %   
QB    REC                                                                     
12    1         0.0   0.0     1.0   1.0     0.0   1.0     0.0   0.0     0.0   
      2         0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
      3         0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
      4         1.0   2.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
      15        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
      80        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     1.0   
      84        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
      88        0.0   0.0     0.0   0.0     0.0   0.0     1.0   1.0     0.0   
FISH  12        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
RACE  12        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
SNAG  POST      0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
UNION 12        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   

O COVERAGE                                       HONDA                      \
ZONE THROWN       FLD SHT       MID INT       BND DEEP       BND SHT         
            count       % count       % count        % count       % count   
QB    REC                                                                    
12    1       0.0     0.0   1.0     0.0   0.0      0.0   0.0     0.0   0.0   
      2       0.0     0.0   0.0     0.0   1.0      0.0   0.0     0.0   0.0   
      3       0.0     0.0   0.0     0.0   0.0      0.0   0.0     0.0   0.0   
      4       0.0     0.0   0.0     0.0   0.0      0.0   0.0     1.0   1.0   
      15      0.0     0.0   0.0     0.0   0.0      0.0   0.0     0.0   0.0   
      80      1.0     0.0   0.0     0.0   0.0      0.0   0.0     0.0   0.0   
      84      0.0     0.0   0.0     0.0   1.0      0.0   1.0     0.0   0.0   
      88      0.0     0.0   0.0     0.0   0.0      0.0   1.0     0.0   0.0   
FISH  12      0.0     0.0   0.0     0.0   0.0      0.0   0.0     0.0   0.0   
RACE  12      0.0     0.0   0.0     0.0   0.0      0.0   0.0     0.0   0.0   
SNAG  POST    0.0     0.0   0.0     0.0   0.0      0.0   0.0     0.0   0.0   
UNION 12      0.0     0.0   0.0     0.0   0.0      0.0   0.0     0.0   0.0   

O COVERAGE                                            IRONMAN                \
ZONE THROWN FLD INT       FLD SHT       MID SHT       BND SHT       FLD SHT   
                  % count       % count       % count       % count       %   
QB    REC                                                                     
12    1         0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
      2         0.0   0.0     1.0   1.0     0.0   0.0     0.0   0.0     1.0   
      3         0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     1.0   
      4         0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
      15        0.0   0.0     1.0   1.0     0.0   0.0     0.0   0.0     0.0   
      80        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
      84        0.0   1.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
      88        0.0   0.0     1.0   1.0     0.0   1.0     1.0   1.0     0.0   
FISH  12        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
RACE  12        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
SNAG  POST      0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   
UNION 12        0.0   0.0     0.0   0.0     0.0   0.0     0.0   0.0     0.0   

O COVERAGE                                     MUG       PIRATE        \
ZONE THROWN       MID INT       MID SHT         12            1         
            count       % count       % count    % count      % count   
QB

In [80]:
styled_pivot = pivot.style \
    .set_caption("Completion Rate by QB–Receiver Duo vs Coverage & Zone Thrown") \
    .set_properties(**{
        'background-color': '#222222',
        'color': 'white',
        'border': '1px solid #555',
        'text-align': 'center'
    }) \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('color', 'white'),
            ('font-size', '16px'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'),
            ('color', 'white'),
            ('border', '1px solid #555'),
            ('font-weight', 'bold'),
            ('text-align', 'center')
        ]}
    ]) \
    .format("{:.1f}")

styled_pivot

### Report 10

In [81]:
# Step 1: Filter for run plays only
run_df = df[df['g R/P'] == 'R'].copy()

# Step 2: Create gain buckets
# Attempt to convert 'G GAIN' column to numeric, coercing errors
run_df['g GAIN'] = pd.to_numeric(run_df['g GAIN'], errors='coerce')

def classify_gain(gain):
    
    if gain <= 0:
        return '0 or less'
    elif 1 <= gain <= 4:
        return '1 – 4'
    elif 5 <= gain <= 10:
        return '5 – 10'
    else:
        return '11 +'

run_df['Gain Bucket'] = run_df['g GAIN'].apply(classify_gain)

# Step 3: Assign OL % per bucket
bucket_ol_pct = {
    '0 or less': 1.20,
    '1 – 4': 1.00,
    '5 – 10': 0.50,
    '11 +': 0.30
}
run_df['OL %'] = run_df['Gain Bucket'].map(bucket_ol_pct)

# Step 4: Compute OL and RB yardage
run_df['OL Yards'] = run_df['g GAIN'] * run_df['OL %']
run_df['RB Yards'] = run_df['g GAIN'] - run_df['OL Yards']

# Step 5: Summarize
summary = run_df.groupby('Gain Bucket').agg(
    Num_Runs=('g GAIN', 'count'),
    Total_Yards=('g GAIN', 'sum'),
    OL_Yards=('OL Yards', 'sum'),
    RB_Yards=('RB Yards', 'sum')
).reset_index()

summary

,Gain Bucket,Num_Runs,Total_Yards,OL_Yards,RB_Yards
0,0 or less,3,-12.0,-14.4,2.4
1,1 – 4,6,20.0,20.0,0.0
2,11 +,3,47.0,14.1,32.9
3,5 – 10,5,31.0,15.5,15.5


In [82]:
styled_summary = summary.style \
    .set_caption("Run Yardage Attribution by Gain Bucket") \
    .set_properties(**{
        'background-color': '#222222',
        'color': 'white',
        'border': '1px solid #555',
        'text-align': 'center'
    }) \
    .set_table_styles([
        {'selector': 'caption', 'props': [
            ('color', 'white'),
            ('font-size', '16px'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('padding-bottom', '10px')
        ]},
        {'selector': 'th', 'props': [
            ('background-color', '#444444'),
            ('color', 'white'),
            ('border', '1px solid #555'),
            ('font-weight', 'bold'),
            ('text-align', 'center')
        ]}
    ]) \
    .format({
        'Num_Runs': '{:,.0f}',
        'Total_Yards': '{:,.1f}',
        'OL_Yards': '{:,.1f}',
        'RB_Yards': '{:,.1f}'
    })

styled_summary

,Gain Bucket,Num_Runs,Total_Yards,OL_Yards,RB_Yards
0,0 or less,3,-12.0,-14.4,2.4
1,1 – 4,6,20.0,20.0,0.0
2,11 +,3,47.0,14.1,32.9
3,5 – 10,5,31.0,15.5,15.5
